In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
import pyabc
import numpy as np
import elfi # install cython -> conda install cython to avoid fallback to numpy
import scipy.stats as ss
import pylab as plt
from pyabc.plots import plot_marginals
%matplotlib notebook

### The simulation model

In [ ]:
def simulator(mu, sigma, batch_size=1, random_state=None):
    mu, sigma = np.atleast_1d(mu, sigma)
    return ss.norm.rvs(mu[:, None], sigma[:, None], size=(batch_size, 30), random_state=random_state)

def mean(y):
    return np.mean(y, axis=1)

def var(y):
    return np.var(y, axis=1)

### Observed data

In [ ]:
# Set the generating parameters that we will try to infer
mean0 = 1
std0 = 3

# Generate some data (using a fixed seed here)
np.random.seed(20170525)
y0 = simulator(mean0, std0)
print(y0)

### Own class

In [ ]:
# setup
mu = pyabc.Prior('unif', -2,4, name="mu")
sigma = pyabc.Prior('unif', 1,4, name="sigma")

rej_samp = pyabc.RejectionSampler(
    priors=[mu, sigma], 
    simulator=simulator, 
    summaries=[mean, var],
    observation=y0
)


smc_samp = pyabc.SMCSampler(
    priors=[mu, sigma], 
    simulator=simulator, 
    summaries=[mean, var],
    observation=y0
)

mcmc_samp = pyabc.MCMCSampler(
    priors=[mu, sigma], 
    simulator=simulator, 
    summaries=[mean, var],
    observation=y0
)


abcde_samp = pyabc.ABCDESampler(
    priors=[mu, sigma], 
    simulator=simulator, 
    summaries=[mean, var],
    observation=y0
)

In [ ]:
rej_samp.sample(nr_samples=100, threshold=.1)

In [ ]:
smc_samp.sample(nr_samples=100, thresholds= [5,1,.5,.1])

In [ ]:
mcmc_samp.sample(nr_samples=100, threshold= .1, step_size = [1,1])

In [ ]:
abcde_samp.sample(nr_samples=100, nr_groups = 10, nr_iter = 200, burn_in = 100, alpha = 0.1, beta = 0.1, kappa = 0.9)

In [ ]:
bolfi.sample(nr_samples=10000, threshold=0.01, n_chains=10)

In [ ]:
plot_marginals(rej_samp, kde=True, normed=True)
plot_marginals(mcmc_samp, kde=True, normed=True)
plot_marginals(smc_samp, kde=True, normed=True)
plot_marginals(abcde_samp, kde=True, normed=True)
plot_marginals(bolfi, kde=True, normed=True)

## ELFI

In [ ]:
# Prior
elfi.new_model() # important to reset, otherwise rerun the code will produce two nodes for S1, S2 and so on
mu = elfi.Prior('uniform', -2, 4)
sigma = elfi.Prior('uniform', 1, 4)

# Add the simulator node and observed data to the model
sim = elfi.Simulator(simulator, mu, sigma, observed=y0)

# Add summary statistics to the model
S1 = elfi.Summary(mean, sim, name="mean")
S2 = elfi.Summary(var, sim, name="var")

# Specify distance as euclidean between summary vectors (S1, S2) from simulated and
# observed data
d = elfi.Distance('euclidean', S1, S2)

In [ ]:
# Plot the complete model (requires graphviz)
elfi.draw(d)

In [ ]:
rej = elfi.Rejection(d, batch_size=10000, seed=30052017)
%time res = rej.sample(1000, threshold=.5)
print(res)

In [ ]:
res.plot_marginals()